In [42]:
import MetaTrader5 as mt5
import pandas as pd
from datetime import datetime, timedelta
import pytz
import numpy as np # Importamos numpy

print("Librerías importadas correctamente.")

Librerías importadas correctamente.


In [62]:
# --- Celda de Generación de Datos Sintéticos (Versión H1) ---

def generar_datos_sinteticos(dias=365 * 3, precio_inicial=1.10000): # Generamos 3 años de datos H1
    """Genera un DataFrame de velas OHLC de 1 HORA con 5 decimales."""
    print(f"Generando {dias} días de datos sintéticos H1...")
    
    # --- CAMBIO CLAVE: FRECUENCIA A '1H' ---
    fechas = pd.date_range(start='2021-01-01', periods=dias * 24, freq='1H')
    n_velas = len(fechas)
    
    # Hacemos los movimientos un poco más grandes para simular la volatilidad horaria
    movimientos = np.random.randn(n_velas) * 0.0005 
    precios_cierre = precio_inicial * (1 + movimientos).cumprod()
    
    df = pd.DataFrame(index=fechas)
    df['Close'] = precios_cierre
    df['Open'] = df['Close'].shift(1).fillna(precio_inicial)
    df['High'] = df[['Open', 'Close']].max(axis=1) + np.random.uniform(0, 0.0010, n_velas)
    df['Low'] = df[['Open', 'Close']].min(axis=1) - np.random.uniform(0, 0.0010, n_velas)
    
    for col in ['Open', 'High', 'Low', 'Close']:
        df[col] = df[col].round(5)
    
    df.rename(columns={'time': 'Time', 'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close'}, inplace=True)
    
    print("Generación completa.")
    return df

# Generamos los nuevos datos H1
df_historico_h1 = generar_datos_sinteticos()

df_historico_h1.head()

Generando 1095 días de datos sintéticos H1...
Generación completa.


C:\Users\Santiago\AppData\Local\Temp\ipykernel_30296\1770117071.py:8: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  fechas = pd.date_range(start='2021-01-01', periods=dias * 24, freq='1H')


,Close,Open,High,Low
2021-01-01 00:00:00,1.10002,1.10000,1.10092,1.09963
2021-01-01 01:00:00,1.09918,1.10002,1.10044,1.09827
2021-01-01 02:00:00,1.09841,1.09918,1.09980,1.09816
2021-01-01 03:00:00,1.09761,1.09841,1.09871,1.09676
2021-01-01 04:00:00,1.09816,1.09761,1.09846,1.09687


In [23]:
# --- PARÁMETROS CON FECHAS HISTÓRICAS FIJAS Y SEGURAS ---
instrumento = "EURUSD"
timeframe = mt5.TIMEFRAME_M1
timezone = pytz.timezone("Etc/UTC")

# Usamos un rango de fechas del pasado que garantizamos que existe.
fecha_desde = datetime(2023, 1, 1, tzinfo=timezone)
fecha_hasta = datetime(2023, 3, 31, tzinfo=timezone)

print(f"Rango de fechas seleccionado: desde {fecha_desde.strftime('%Y-%m-%d')} hasta {fecha_hasta.strftime('%Y-%m-%d')}")

Rango de fechas seleccionado: desde 2023-01-01 hasta 2023-03-31


In [24]:
print(f"Descargando datos para {instrumento} desde {fecha_desde} hasta {fecha_hasta}...")

# Pedimos los datos históricos a MetaTrader 5
rates = mt5.copy_rates_range(instrumento, timeframe, fecha_desde, fecha_hasta)

# Cerramos la conexión con MT5
mt5.shutdown()
print("Conexión con MetaTrader 5 cerrada.")

# --- NUEVA VERIFICACIÓN DE ERRORES ---
# Comprobamos si la descarga falló (si rates es None)
if rates is None:
    print("\nERROR: No se pudieron descargar los datos.")
    print("Posibles causas:")
    print("1. Tu bróker no tiene datos históricos para el rango de fechas solicitado.")
    print("2. Revisa la conexión con la terminal de MT5.")
else:
    print(f"\nDescarga completa. Se obtuvieron {len(rates)} velas.")

Descargando datos para EURUSD desde 2023-01-01 00:00:00+00:00 hasta 2023-03-31 00:00:00+00:00...
Conexión con MetaTrader 5 cerrada.

ERROR: No se pudieron descargar los datos.
Posibles causas:
1. Tu bróker no tiene datos históricos para el rango de fechas solicitado.
2. Revisa la conexión con la terminal de MT5.


In [30]:
pip install backtesting

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [58]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import pandas as pd
import pandas_ta as ta

# --- Estrategia v5.0 - CRUCE DE MEDIAS MÓVILES ---

class MovingAverageCrossover(Strategy):
    # Definimos los períodos de nuestras dos medias móviles
    fast_ema_period = 50
    slow_ema_period = 200

    def init(self):
        # Calculamos ambas medias móviles para todo el historial de datos
        self.ema_fast = self.I(ta.ema, pd.Series(self.data.Close), length=self.fast_ema_period)
        self.ema_slow = self.I(ta.ema, pd.Series(self.data.Close), length=self.slow_ema_period)
        print("Medias móviles calculadas.")

    def next(self):
        # --- LÓGICA DE TRADING DE TENDENCIA ---
        
        # Condición de Compra: si la EMA rápida cruza por encima de la EMA lenta
        if crossover(self.ema_fast, self.ema_slow):
            # Si hay una posición de venta abierta, la cerramos primero
            if self.position.is_short:
                self.position.close()
            # Abrimos una nueva posición de compra
            self.buy()

        # Condición de Venta: si la EMA rápida cruza por debajo de la EMA lenta
        elif crossover(self.ema_slow, self.ema_fast):
            # Si hay una posición de compra abierta, la cerramos primero
            if self.position.is_long:
                self.position.close()
            # Abrimos una nueva posición de venta
            self.sell()

In [63]:
# --- Celda de Simulación Simple (con datos H1) ---

# Usamos el DataFrame H1 para probar la estrategia
bt = Backtest(df_historico_h1, MovingAverageCrossover, 
              cash=10000, commission=.0002,
              finalize_trades=True)

stats = bt.run()
print(stats)

Medias móviles calculadas.
Start                     2021-01-01 00:00:00
End                       2023-12-31 23:00:00
Duration                   1094 days 23:00:00
Exposure Time [%]                    98.68341
Equity Final [$]                   7955.99699
Equity Peak [$]                   10107.83945
Commissions [$]                     730.60899
Return [%]                          -20.44003
Buy & Hold Return [%]                -2.69675
Return (Ann.) [%]                    -7.33874
Volatility (Ann.) [%]                 4.42986
CAGR [%]                             -7.33901
Sharpe Ratio                         -1.65665
Sortino Ratio                        -2.03496
Calmar Ratio                         -0.32639
Alpha [%]                           -20.55733
Beta                                 -0.04349
Max. Drawdown [%]                   -22.48432
Avg. Drawdown [%]                    -1.61954
Max. Drawdown Duration      914 days 09:00:00
Avg. Drawdown Duration       56 days 21:00:00
# Trade

In [41]:
# Generamos el gráfico interactivo del backtest
bt.plot()

C:\Users\Santiago\Documents\supertrax\backend\venv\Lib\site-packages\backtesting\_plotting.py:141: UserWarning: Data contains too many candlesticks to plot; downsampling to '1h'. See `Backtest.plot(resample=...)`
  warnings.warn(f"Data contains too many candlesticks to plot; downsampling to {freq!r}. "


GridPlot(id='p2288', ...)

In [50]:
# --- Celda de Optimización ---

print("Iniciando optimización de la estrategia...")

# Le pasamos nuestra estrategia y definimos los rangos que queremos probar para cada parámetro.
# Probaremos Stop Loss desde 5 hasta 25 pips, en incrementos de 5.
# Probaremos Take Profit desde 10 hasta 60 pips, en incrementos de 10.
stats_opt, heatmap = bt.optimize(
    sl_pips=range(5, 30, 5),      # SL: 5, 10, 15, 20, 25
    tp_pips=range(10, 70, 10),     # TP: 10, 20, 30, 40, 50, 60
    maximize='Equity Final [$]', # Le decimos que queremos encontrar la combinación que dé el mayor equity final.
    constraint=lambda param: param.tp_pips > param.sl_pips # Añadimos una regla: el TP siempre debe ser mayor que el SL.
)

print("\nOptimización completa.")

# Imprimimos las estadísticas de la MEJOR combinación encontrada.
print("\n--- MEJORES PARÁMETROS ENCONTRADOS ---")
print(stats_opt)

# Imprimimos los valores específicos de los mejores parámetros.
print("\n--- VALORES DE LOS PARÁMETROS ---")
print(stats_opt._strategy)

Iniciando optimización de la estrategia...


C:\Users\Santiago\Documents\supertrax\backend\venv\Lib\site-packages\backtesting\backtesting.py:1624: RuntimeWarning: If you want to use multi-process optimization with `multiprocessing.get_start_method() == 'spawn'` (e.g. on Windows),set `backtesting.Pool = multiprocessing.Pool` (or of the desired context) and hide `bt.optimize()` call behind a `if __name__ == '__main__'` guard. Currently using thread-based paralellism, which might be slightly slower for non-numpy / non-GIL-releasing code. See https://github.com/kernc/backtesting.py/issues/1256
  output = _optimize_grid()


Inicializando la estrategia y calculando fractales para todo el historial...
Inicializando la estrategia y calculando fractales para todo el historial...
Inicializando la estrategia y calculando fractales para todo el historial...
Inicializando la estrategia y calculando fractales para todo el historial...
Cálculo de fractales completo.
Cálculo de fractales completo.
Cálculo de fractales completo.
Cálculo de fractales completo.


C:\Users\Santiago\Documents\supertrax\backend\venv\Lib\site-packages\backtesting\backtesting.py:1637: UserWarning: Some trades remain open at the end of backtest. Use `Backtest(..., finalize_trades=True)` to close them and include them in stats.
  for stats in (bt.run(**params)
C:\Users\Santiago\Documents\supertrax\backend\venv\Lib\site-packages\backtesting\backtesting.py:1637: UserWarning: Some trades remain open at the end of backtest. Use `Backtest(..., finalize_trades=True)` to close them and include them in stats.
  for stats in (bt.run(**params)
C:\Users\Santiago\Documents\supertrax\backend\venv\Lib\site-packages\backtesting\backtesting.py:1637: UserWarning: Some trades remain open at the end of backtest. Use `Backtest(..., finalize_trades=True)` to close them and include them in stats.
  for stats in (bt.run(**params)


Inicializando la estrategia y calculando fractales para todo el historial...
Inicializando la estrategia y calculando fractales para todo el historial...
Inicializando la estrategia y calculando fractales para todo el historial...
Inicializando la estrategia y calculando fractales para todo el historial...
Cálculo de fractales completo.
Cálculo de fractales completo.
Cálculo de fractales completo.
Cálculo de fractales completo.


...

Inicializando la estrategia y calculando fractales para todo el historial...
Inicializando la estrategia y calculando fractales para todo el historial...
Inicializando la estrategia y calculando fractales para todo el historial...
Cálculo de fractales completo.
Cálculo de fractales completo.
Cálculo de fractales completo.
Cálculo de fractales completo.


C:\Users\Santiago\Documents\supertrax\backend\venv\Lib\site-packages\backtesting\backtesting.py:1637: UserWarning: Some trades remain open at the end of backtest. Use `Backtest(..., finalize_trades=True)` to close them and include them in stats.
  for stats in (bt.run(**params)
C:\Users\Santiago\Documents\supertrax\backend\venv\Lib\site-packages\backtesting\backtesting.py:1637: UserWarning: Some trades remain open at the end of backtest. Use `Backtest(..., finalize_trades=True)` to close them and include them in stats.
  for stats in (bt.run(**params)
C:\Users\Santiago\Documents\supertrax\backend\venv\Lib\site-packages\backtesting\backtesting.py:1637: UserWarning: Some trades remain open at the end of backtest. Use `Backtest(..., finalize_trades=True)` to close them and include them in stats.
  for stats in (bt.run(**params)


Inicializando la estrategia y calculando fractales para todo el historial...
Cálculo de fractales completo.


C:\Users\Santiago\Documents\supertrax\backend\venv\Lib\site-packages\backtesting\backtesting.py:1545: UserWarning: Some trades remain open at the end of backtest. Use `Backtest(..., finalize_trades=True)` to close them and include them in stats.
  stats = self.run(**dict(zip(heatmap.index.names, best_params)))


ValueError: too many values to unpack (expected 2)

In [54]:
# --- Celda de Optimización (Versión 2.0 - Corregida) ---

# 1. Configuramos el backtest, AÑADIENDO 'finalize_trades=True'
# Esto asegura que cada simulación termine de forma limpia.
bt = Backtest(df_historico, FractalBreakoutStrategy, 
              cash=10000, commission=.0002,
              finalize_trades=True) # <--- ¡LA CORRECCIÓN CLAVE ESTÁ AQUÍ!

print("Iniciando optimización de la estrategia...")

# 2. Ejecutamos la optimización. Ahora no debería fallar.
stats_opt = bt.optimize(
    sl_pips=range(5, 30, 5),
    tp_pips=range(10, 70, 10),
    maximize='Equity Final [$]',
    constraint=lambda param: param.tp_pips > param.sl_pips
)

print("\nOptimización completa.")

# 3. Imprimimos los resultados de la mejor combinación.
print("\n--- MEJORES PARÁMETROS ENCONTRADOS ---")
print(stats_opt)

print("\n--- VALORES DE LOS PARÁMETROS ---")
print(stats_opt._strategy)

# 4. Generamos el gráfico de la mejor simulación.
bt.plot(plot_volume=False, plot_pl=False)

Iniciando optimización de la estrategia...


C:\Users\Santiago\Documents\supertrax\backend\venv\Lib\site-packages\backtesting\backtesting.py:1624: RuntimeWarning: If you want to use multi-process optimization with `multiprocessing.get_start_method() == 'spawn'` (e.g. on Windows),set `backtesting.Pool = multiprocessing.Pool` (or of the desired context) and hide `bt.optimize()` call behind a `if __name__ == '__main__'` guard. Currently using thread-based paralellism, which might be slightly slower for non-numpy / non-GIL-releasing code. See https://github.com/kernc/backtesting.py/issues/1256
  output = _optimize_grid()


Filtro de tendencia EMA(200) calculado.
Filtro de tendencia EMA(200) calculado.
Filtro de tendencia EMA(200) calculado.Filtro de tendencia EMA(200) calculado.

Filtro de tendencia EMA(200) calculado.
Filtro de tendencia EMA(200) calculado.
Filtro de tendencia EMA(200) calculado.
Filtro de tendencia EMA(200) calculado.
Filtro de tendencia EMA(200) calculado.
Filtro de tendencia EMA(200) calculado.
Filtro de tendencia EMA(200) calculado.
Filtro de tendencia EMA(200) calculado.
Filtro de tendencia EMA(200) calculado.
Filtro de tendencia EMA(200) calculado.
Filtro de tendencia EMA(200) calculado.
Filtro de tendencia EMA(200) calculado.
Filtro de tendencia EMA(200) calculado.
Filtro de tendencia EMA(200) calculado.
Filtro de tendencia EMA(200) calculado.
Filtro de tendencia EMA(200) calculado.
Filtro de tendencia EMA(200) calculado.
Filtro de tendencia EMA(200) calculado.
Filtro de tendencia EMA(200) calculado.
Filtro de tendencia EMA(200) calculado.
Filtro de tendencia EMA(200) calculado.


C:\Users\Santiago\Documents\supertrax\backend\venv\Lib\site-packages\backtesting\_plotting.py:141: UserWarning: Data contains too many candlesticks to plot; downsampling to '1h'. See `Backtest.plot(resample=...)`
  warnings.warn(f"Data contains too many candlesticks to plot; downsampling to {freq!r}. "


GridPlot(id='p2835', ...)